In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
# Distribuciones
from scipy.stats import uniform, lognorm, norm 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import f1_score
import tensorflow_hub as hub 
from google.colab import drive
from sklearn.externals import joblib 
drive.mount('/content/drive')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/My Drive/Factored project/Models/newdata.csv')
data = data.drop(columns=['Unnamed: 0'], axis=1)
Y = data['target']
X = data.drop(columns=['target'], axis=1)

In [ ]:
def Embedding(X):
  embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
  X = embed(X)
  X = X.numpy()
  return X 

In [ ]:
model = LogisticRegression(max_iter=500, solver='saga', random_state=40)
dist = dict(C=uniform(loc=0, scale=4), penalty=['l2', 'l1'])
RS_LR = RandomizedSearchCV(model, dist, random_state=0, scoring='f1', n_iter=20)
Resultados = {'Set':['Train', 'Test']}
for i in X.columns:
  x_em = Embedding(X.loc[:,i])
  # Partición de los datos
  x_train, x_test, y_train, y_test = train_test_split(x_em, Y, test_size=0.2, random_state=40)
  # Buscar hiperparámetros que maximizan la metrica f1
  search = RS_LR.fit(x_train, y_train)
  var = 'model_LR_{}'.format(i)
  #joblib.dump(search, '/content/drive/My Drive/Factored project/Hyperparameter/{}.pkl'.format(var))
  y_pre = search.predict(x_test)
  Resultados[i] = [search.best_score_, f1_score(y_test, y_pre)]

In [ ]:
# R_hyp = pd.DataFrame(data=Resultados)
# R_hyp.to_csv('/content/drive/My Drive/Factored project/Hyperparameter/R_LR_whyp.csv')
# R_hyp.head()

In [ ]:
R_hyp = pd.DataFrame(data=Resultados)
#R_hyp.to_csv('/content/drive/My Drive/Factored project/Hyperparameter/R_LR_whyp.csv')
R_hyp.head()

,Set,text,NoRuido,PRE,URLS,HTML,EMOJIS,TODO
0,Train,0.769669,0.768300,0.763665,0.768300,0.769669,0.769669,0.763665
1,Test,0.779079,0.783217,0.759907,0.783217,0.779079,0.779079,0.759907


In [ ]:
RR = pd.read_csv('/content/drive/My Drive/Factored project/Hyperparameter/R_LR_whyp.csv')
RR.head()

,Unnamed: 0,Set,text,NoRuido,PRE,URLS,HTML,EMOJIS,TODO
0,0,Train,0.769669,0.768300,0.763406,0.768300,0.769669,0.769669,0.763406
1,1,Test,0.779079,0.783217,0.767551,0.783217,0.779079,0.779079,0.767551


In [ ]:
RRR = pd.read_csv('/content/drive/My Drive/Factored project/Models/results_RL.csv')
RRR.head()

,Unnamed: 0,Embedding,text,NoRuido,PRE,URLS,HTML,EMOJIS,TODO
0,0,BOW,0.770465,0.767886,0.758509,0.767886,0.770465,0.770465,0.758509
1,1,TF IDF,0.751055,0.754181,0.738832,0.754181,0.751055,0.751055,0.738832
2,2,Tensorflow hub,0.782541,0.779688,0.765759,0.779688,0.782541,0.782541,0.765759


In [ ]:
model_sh = joblib.load('/content/drive/My Drive/Factored project/Models/model_1_ttext_2.pkl')
model_ch = joblib.load('/content/drive/My Drive/Factored project/Models/')